<a href="https://colab.research.google.com/github/gauthamhere/cycGan/blob/master/GauthamK_EIP3P2_anime_FaceGAN_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#Downloading and installing keras-contrib inorder to use keras instance normalization
!pip install git+https://www.github.com/keras-team/keras-contrib.git

  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-kd4irief
  Running command git clone -q https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-kd4irief
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-cp36-none-any.whl size=101066 sha256=163a5dd5a6c9cb8b14fe9acd1c00c6b05fbed174f32be48b9b62ae4337718918
  Stored in directory: /tmp/pip-ephem-wheel-cache-spgz8rox/wheels/11/27/c8/4ed56de7b55f4f61244e2dc6ef3cdbaff2692527a2ce6502ba
Successfully built keras-contrib


In [0]:
from keras.preprocessing.image import ImageDataGenerator
import scipy
from glob import glob
import numpy as np
import os
import tensorflow as tf
from keras.engine.topology import Layer
from keras.engine import InputSpec

from keras.models import Model
from keras.layers import Input, Conv2D

from __future__ import print_function, division

from keras_contrib.layers.normalization.instancenormalization import InstanceNormalization
from keras.layers import Input, Dropout, Activation, Add
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.models import Model
from keras.optimizers import Adam
import numpy as np
import datetime
import matplotlib.pyplot as plt
import scipy.misc


class DataLoader:
    def __init__(self, dataset_name, img_res=(128, 128)):
        self.dataset_name = dataset_name
        self.img_res = img_res

    def load_data(self, domain, batch_size=1, is_testing=False):

        data_type = "train%s" % domain if not is_testing else "test%s" % domain
        path = glob('%s/%s/*' % (self.dataset_name, data_type))

        batch_images = np.random.choice(path, size=batch_size)

        imgs = []
        for img_path in batch_images:
            img = self.imread(img_path)
            if not is_testing:
                img = scipy.misc.imresize(img, self.img_res)

                if np.random.random() > 0.5:
                    img = np.fliplr(img)
            else:
                img = scipy.misc.imresize(img, self.img_res)
            imgs.append(img)

        imgs = np.array(imgs)/127.5 - 1.

        return imgs

    def load_k_data(self, domain, image_number=10, is_testing=False):
        data_type = "train%s" % domain if not is_testing else "test%s" % domain
        path = glob('%s/%s/*' % (self.dataset_name, data_type))

        images = np.array(path)
        if image_number < len(path):
            images = images[:image_number]

        imgs = []
        for img_path in images:
            img = self.imread(img_path)
            if not is_testing:
                img = scipy.misc.imresize(img, self.img_res)

                if np.random.random() > 0.5:
                    img = np.fliplr(img)
            else:
                img = scipy.misc.imresize(img, self.img_res)
            imgs.append(img)

        imgs = np.array(imgs) / 127.5 - 1.

        return imgs

    def load_batch(self, batch_size=1, is_testing=False, aug=False):
        data_type = "train" if not is_testing else "test"
 
        path_A = glob('%s/%sA/*' % (self.dataset_name, data_type))
        path_B = glob('%s/%sB/*' % (self.dataset_name, data_type))

        if not is_testing and aug:
            datagen = ImageDataGenerator(
                zoom_range=0.2,
                rotation_range=15,
                width_shift_range=0.1,
                height_shift_range=0.1,
                horizontal_flip=True,
                brightness_range=[0.5, 1.5]
            )

        self.n_batches = int(min(len(path_A), len(path_B)) / batch_size)
        total_samples = self.n_batches * batch_size

        # Sample n_batches * batch_size from each path list so that model sees all
        # samples from both domains
        path_A = np.random.choice(path_A, total_samples, replace=False)
        path_B = np.random.choice(path_B, total_samples, replace=False)

        for i in range(self.n_batches-1):
            batch_A = path_A[i*batch_size:(i+1)*batch_size]
            batch_B = path_B[i*batch_size:(i+1)*batch_size]
            imgs_A, imgs_B = [], []
            for img_A, img_B in zip(batch_A, batch_B):
                img_A = self.imread(img_A)
                img_B = self.imread(img_B)

                img_A = scipy.misc.imresize(img_A, self.img_res)
                img_B = scipy.misc.imresize(img_B, self.img_res)

                # Data augmenting
                if not is_testing and aug:
                    img_A = datagen.random_transform(img_A)
                    img_B = datagen.random_transform(img_B)

                imgs_A.append(img_A)
                imgs_B.append(img_B)

            imgs_A = np.array(imgs_A)/127.5 - 1.
            imgs_B = np.array(imgs_B)/127.5 - 1.

            yield imgs_A, imgs_B

    def load_img(self, path):
        img = self.imread(path)
        img = scipy.misc.imresize(img, self.img_res)
        img = img/127.5 - 1.
        return img[np.newaxis, :, :, :]

    def imread(self, path):
        return scipy.misc.imread(path, mode='RGB').astype(np.float)

Using TensorFlow backend.


In [0]:
class ReflectionPadding2D(Layer):
    def __init__(self, padding, **kwargs):
        self.padding = tuple(padding)
        self.input_spec = [InputSpec(ndim=4)]
        super(ReflectionPadding2D, self).__init__(**kwargs)

    def compute_output_shape(self, input_shape):
        shape = (
            input_shape[0],
            input_shape[1] + 2 * self.padding[0],
            input_shape[2] + 2 * self.padding[1],
            input_shape[3]
        )
        return shape

    def call(self, x, mask=None):
        width_pad, height_pad = self.padding
        return tf.pad(
            x,
            [[0, 0], [height_pad, height_pad], [width_pad, width_pad], [0, 0]],
            'REFLECT'
        )


nn_input = Input((128, 128, 3))
reflect_pad = ReflectionPadding2D(padding=(3, 3))(nn_input)
conv2d = Conv2D(32, kernel_size=7, strides=1, padding="valid")(reflect_pad)
model = Model(nn_input, conv2d)
model.summary()

W0813 06:01:39.287622 140582191855488 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0813 06:01:39.323106 140582191855488 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0813 06:01:39.332142 140582191855488 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 128, 128, 3)       0         
_________________________________________________________________
reflection_padding2d_1 (Refl (None, 134, 134, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 128, 32)      4736      
Total params: 4,736
Trainable params: 4,736
Non-trainable params: 0
_________________________________________________________________


## Dataset

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
from shutil import copyfile
copyfile("gdrive/My Drive/anime_dataset/anime_face.zip","anime_face.zip")

'anime_face.zip'

In [0]:
!ls

anime_face.zip	gdrive	sample_data


In [0]:
!unzip -q anime_face.zip

In [0]:
!ls anime_face/

anime  human


Splitting the data to train and test

In [0]:
!mkdir anime_traintest

In [0]:
!mkdir anime_traintest/trainanime
!mkdir anime_traintest/trainhuman
!mkdir anime_traintest/testanime
!mkdir anime_traintest/testhuman

In [0]:
import random
import os

In [0]:
#specs
anime_fnames = []
for fnames in os.listdir("anime_face/anime"):
  anime_fnames.append(fnames)

In [0]:
len(anime_fnames)

5600

In [0]:
# anime_fnames
anime_fnames.sort()  # make sure that the filenames have a fixed order before shuffling
random.seed(230)
random.shuffle(anime_fnames) # shuffles the ordering of filenames (deterministic given the chosen seed)

split_1 = int(0.8 * len(anime_fnames))
train_filenames = anime_fnames[:split_1]
test_filenames = anime_fnames[split_1:]

In [0]:
len(train_filenames)

4480

In [0]:
len(test_filenames)

1120

In [0]:
for sfnames in train_filenames:
  copyfile(os.path.join("anime_face/anime",sfnames),os.path.join("anime_traintest/trainanime",sfnames))

In [0]:
for sfnames in test_filenames:
  copyfile(os.path.join("anime_face/anime",sfnames),os.path.join("anime_traintest/testanime",sfnames))

In [0]:
#human
human_fnames = []
for wfnames in os.listdir("anime_face/human"):
  human_fnames.append(wfnames)

In [0]:
len(human_fnames)

5137

In [0]:
# human_fnames
human_fnames.sort()  # make sure that the filenames have a fixed order before shuffling
random.seed(230)
random.shuffle(human_fnames) # shuffles the ordering of filenames (deterministic given the chosen seed)

split_1 = int(0.8 * len(human_fnames))
wtrain_filenames = human_fnames[:split_1]
wtest_filenames = human_fnames[split_1:]

In [0]:
len(wtrain_filenames)

4109

In [0]:
len(wtest_filenames)

1028

In [0]:
for sfnames in wtrain_filenames:
  copyfile(os.path.join("anime_face/human",sfnames),os.path.join("anime_traintest/trainhuman",sfnames))

In [0]:
for sfnames in wtest_filenames:
  copyfile(os.path.join("anime_face/human",sfnames),os.path.join("anime_traintest/testhuman",sfnames))

In [0]:
!zip -r -q anime_traintest.zip anime_traintest

In [0]:
copyfile("anime_traintest.zip","gdrive/My Drive/anime_dataset/anime_traintest.zip")

'gdrive/My Drive/anime_dataset/anime_traintest.zip'

In [0]:
from shutil import copyfile
copyfile("gdrive/My Drive/anime_dataset/anime_traintest.zip","anime_traintest.zip")

'anime_traintest.zip'

In [0]:
!unzip -q anime_traintest.zip

In [0]:
!ls anime_traintest

testanime  testhuman  trainanime  trainhuman


In [0]:
!mv anime_traintest/trainanime anime_traintest/trainA
!mv anime_traintest/trainhuman anime_traintest/trainB
!mv anime_traintest/testanime anime_traintest/testA
!mv anime_traintest/testhuman anime_traintest/testB

In [0]:
!rm -rf anime_traintest/trainA/61050_2018.jpg

In [0]:
!ls anime_traintest

testA  testB  trainA  trainB


In [0]:
!pip install scipy==1.1.0

     |████████████████████████████████| 31.2MB 63.3MB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: scipy 1.3.0
    Uninstalling scipy-1.3.0:
      Successfully uninstalled scipy-1.3.0


In [0]:
!mkdir models
!mkdir images
!mkdir test_images
!mkdir images/anime_traintest_paper

## Model

In [0]:
class CycleGAN:
    def __init__(self):
        # Input shape
        self.img_rows = 128
        self.img_cols = 128
        self.channels = 3
        self.img_shape = (self.img_rows, self.img_cols, self.channels)

        # Configure data loader
        self.dataset_name = 'anime_traintest'
        self.data_loader = DataLoader(dataset_name=self.dataset_name,
                                      img_res=(self.img_rows, self.img_cols))

        # Calculate output shape of D (PatchGAN)
        patch = int(self.img_rows / 2 ** 4)
        self.disc_patch = (patch, patch, 1)

        # Number of filters in the first layer of G and D
        self.gf = 64
        self.df = 64

        # Loss weights
        self.lambda_cycle = 10.0  # Cycle-consistency loss
        self.lambda_id = 0.1 * self.lambda_cycle  # Identity loss

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminators
        self.d_A = self.build_discriminator()
        self.d_B = self.build_discriminator()
        self.d_A.compile(loss='mse',
                         optimizer=optimizer,
                         metrics=['accuracy'])
        self.d_B.compile(loss='mse',
                         optimizer=optimizer,
                         metrics=['accuracy'])

        # -------------------------
        # Construct Computational
        #   Graph of Generators
        # -------------------------

        # Build the generators
        self.g_AB = self.build_generator()
        self.g_BA = self.build_generator()

        # Input images from both domains
        img_A = Input(shape=self.img_shape)
        img_B = Input(shape=self.img_shape)

        # Translate images to the other domain
        fake_B = self.g_AB(img_A)
        fake_A = self.g_BA(img_B)
        # Translate images back to original domain
        reconstr_A = self.g_BA(fake_B)
        reconstr_B = self.g_AB(fake_A)
        # Identity mapping of images
        img_A_id = self.g_BA(img_A)
        img_B_id = self.g_AB(img_B)

        # For the combined model we will only train the generators
        self.d_A.trainable = False
        self.d_B.trainable = False

        # Discriminators determines validity of translated images
        valid_A = self.d_A(fake_A)
        valid_B = self.d_B(fake_B)

        # Combined model trains generators to fool discriminators
        self.combined = Model(inputs=[img_A, img_B],
                              outputs=[valid_A, valid_B,
                                       reconstr_A, reconstr_B,
                                       img_A_id, img_B_id])
        self.combined.compile(loss=['mse', 'mse',
                                    'mae', 'mae',
                                    'mae', 'mae'],
                              loss_weights=[1, 1,
                                            self.lambda_cycle, self.lambda_cycle,
                                            self.lambda_id, self.lambda_id],
                              optimizer=optimizer)

        self.current_epoch = 0

    def build_generator(self):
        def conv2d(layer_input, filters, f_size=4, stride=2, padding='valid'):
            d = Conv2D(filters, kernel_size=f_size, strides=stride, padding=padding)(layer_input)
            d = InstanceNormalization()(d)
            d = Activation('relu')(d)
            return d

        def res_block(layer_input, filters=256, use_dropout=False):
            y = ReflectionPadding2D(padding=(1, 1))(layer_input)
            y = conv2d(y, filters, 3, 1)
            if use_dropout:
                y = Dropout(0.5)(y)
            y = ReflectionPadding2D(padding=(1, 1))(y)
            y = conv2d(y, filters, 3, 1)
            return Add()([y, layer_input])

        def deconv2d(layer_input, filters, f_size=4, dropout_rate=0):
            u = Conv2DTranspose(filters, kernel_size=f_size, strides=2, padding='same')(layer_input)
            if dropout_rate:
                u = Dropout(dropout_rate)(u)
            u = InstanceNormalization()(u)
            u = Activation('relu')(u)
            return u

        # Image input
        d0 = Input(shape=self.img_shape)
        d1 = ReflectionPadding2D(padding=(3, 3))(d0)
        # c7s1-64
        d1 = conv2d(d1, self.gf, 7, 1)
        # d128
        d2 = conv2d(d1, self.gf * 2, 3, padding='same')
        # d256
        d3 = conv2d(d2, self.gf * 4, 3, padding='same')
        # R256,R256,R256,R256,R256,R256
        r = res_block(d3)
        r = res_block(r)
        r = res_block(r)
        r = res_block(r)
        r = res_block(r)
        r = res_block(r)
        # u128
        u1 = deconv2d(r, self.gf * 2, 3)
        # u64
        u2 = deconv2d(u1, self.gf, 3)
        # c7s1-3
        u3 = ReflectionPadding2D(padding=(3, 3))(u2)
        output_img = Conv2D(self.channels, kernel_size=7, strides=1, activation='tanh')(u3)

        return Model(d0, output_img)

    def build_discriminator(self):
        def d_layer(layer_input, filters, f_size=4, normalization=True):
            d = Conv2D(filters, kernel_size=f_size, strides=2, padding='same')(layer_input)
            if normalization:
                d = InstanceNormalization()(d)
            d = LeakyReLU(alpha=0.2)(d)
            return d

        img = Input(shape=self.img_shape)
        # C64-C128-C256-C512
        d1 = d_layer(img, self.df, normalization=False)
        d2 = d_layer(d1, self.df * 2)
        d3 = d_layer(d2, self.df * 4)
        d4 = d_layer(d3, self.df * 8)
        validity = Conv2D(1, kernel_size=4, strides=1, padding='same')(d4)

        return Model(img, validity)

    def load_network(self, n_epoch_start,path):
        # returns a compiled model identical to the previous one
        self.current_epoch = n_epoch_start+1
        self.combined.load_weights(path)

    def train(self, epochs, batch_size=1, sample_interval=50):
        start_time = datetime.datetime.now()
        print(start_time)

        # Adversarial loss ground truths
        valid = np.ones((batch_size,) + self.disc_patch)
        fake = np.zeros((batch_size,) + self.disc_patch)

        for epoch in range(epochs):
            for batch_i, (imgs_A, imgs_B) in enumerate(self.data_loader.load_batch(batch_size, aug=True)):
                # ----------------------
                #  Train Discriminators
                # ----------------------
                # Translate images to opposite domain
                fake_B = self.g_AB.predict(imgs_A)
                fake_A = self.g_BA.predict(imgs_B)
                # Train the discriminators (original images = real / translated = Fake)
                dA_loss_real = self.d_A.train_on_batch(imgs_A, valid)
                dA_loss_fake = self.d_A.train_on_batch(fake_A, fake)
                dA_loss = 0.5 * np.add(dA_loss_real, dA_loss_fake)
                dB_loss_real = self.d_B.train_on_batch(imgs_B, valid)
                dB_loss_fake = self.d_B.train_on_batch(fake_B, fake)
                dB_loss = 0.5 * np.add(dB_loss_real, dB_loss_fake)
                # Total disciminator loss
                d_loss = 0.5 * np.add(dA_loss, dB_loss)

                # ------------------
                #  Train Generators
                # ------------------

                # Train the generators
                g_loss = self.combined.train_on_batch([imgs_A, imgs_B],
                                                      [valid, valid,
                                                       imgs_A, imgs_B,
                                                       imgs_A, imgs_B])

                elapsed_time = datetime.datetime.now() - start_time

                # Plot the progress
                print(
                    "[Epoch %d/%d] [Batch %d/%d] [D loss: %f, acc: %3d%%] [G loss: %05f, adv: %05f, recon: %05f, id: %05f] time: %s " \
                    % (epoch, epochs,
                       batch_i, self.data_loader.n_batches,
                       d_loss[0], 100 * d_loss[1],
                       g_loss[0],
                       np.mean(g_loss[1:3]),
                       np.mean(g_loss[3:5]),
                       np.mean(g_loss[5:6]),
                       elapsed_time))

                # If at save interval => save generated image samples
                if batch_i % sample_interval == 0:
                    self.sample_images(epoch+self.current_epoch, batch_i)

            if epoch % 10 == 0 and (epoch != 0 or self.current_epoch != 0):
                self.combined.save("models/combined_ep{}.h5".format(
                    self.dataset_name,
                    epoch+self.current_epoch)
                )
                self.combined.save(path+"models/combined_ep_{}.h5".format(
                    epoch+self.current_epoch)
                )

    def sample_images(self, epoch, batch_i):
        r, c = 2, 3
        imgs_A = self.data_loader.load_data(domain="A", batch_size=1, is_testing=True)
        imgs_B = self.data_loader.load_data(domain="B", batch_size=1, is_testing=True)
        # Translate images to the other domain
        fake_B = self.g_AB.predict(imgs_A)
        fake_A = self.g_BA.predict(imgs_B)
        # Translate back to original domain
        reconstr_A = self.g_BA.predict(fake_B)
        reconstr_B = self.g_AB.predict(fake_A)

        gen_imgs = np.concatenate([imgs_A, fake_B, reconstr_A, imgs_B, fake_A, reconstr_B])
        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        titles = ['Original', 'Translated', 'Reconstructed']
        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i, j].imshow(gen_imgs[cnt])
                axs[i, j].set_title(titles[j])
                axs[i, j].axis('off')
                cnt += 1

        fig.savefig("images/{}_paper/{}_{}.png".format(
            self.dataset_name, epoch, batch_i))
        fig.savefig(path+"images/{}_paper/{}_{}.png".format(
            self.dataset_name, epoch, batch_i))
        plt.close()

    def test_k_images(self, im_path="1.jpg",translated_path="t.jpg",output_path="o.jpg"):
        r, c = 1, 3
        
        img_res=(128, 128)

        
        imgs_A = scipy.misc.imread(im_path)
        imgs_A = scipy.misc.imresize(imgs_A, img_res)
        imgs_A = np.array(imgs_A)/127.5 - 1.
        
        
        imgs_A = np.expand_dims(imgs_A,axis=0)
        
        
        

        # Translate images to the other domain
        fake_B = self.g_AB.predict(imgs_A)
#         fake_A = self.g_BA.predict(imgs_B)
        # Translate back to original domain
        reconstr_A = self.g_BA.predict(fake_B)
#         reconstr_B = self.g_AB.predict(fake_A)

        for image_i in range(len(imgs_A)):
            gen_imgs = np.concatenate([
                np.expand_dims(imgs_A[image_i], axis=0),
                np.expand_dims(fake_B[image_i], axis=0),
                np.expand_dims(reconstr_A[image_i], axis=0),
#                 np.expand_dims(imgs_B[image_i], axis=0),
#                 np.expand_dims(fake_A[image_i], axis=0),
#                 np.expand_dims(reconstr_B[image_i], axis=0)
            ])

            # Rescale images 0 - 1
            gen_imgs = 0.5 * gen_imgs + 0.5
          
            translated_img = gen_imgs[1]
            scipy.misc.imsave(translated_path, translated_img)
             
          
          
          
            fig, axs = plt.subplots(1, 3, figsize=(10, 3))
            cnt = 0
            for ax, title_im in zip(axs, ['original', 'translated', 'reconstructed']):
              ax.imshow(gen_imgs[cnt])
              ax.axis('off')
              ax.set_title(title_im.capitalize())
              ax.grid(True)
              cnt+=1
              
            fig.savefig(output_path)
            plt.show()
            plt.close()

## Training _ till 70 epochs - 12 hours 

In [0]:
cyclegan = CycleGAN()
cyclegan.train(epochs=100, batch_size=16, sample_interval=200,path = "gdrive/My Drive/anime/")

## Training after 70th epoch

In [0]:
cyclegan = CycleGAN()
cyclegan.load_network(70,path = "gdrive/My Drive/anime/models/combined_epanime_traintest.h5")
cyclegan.train(epochs=80, batch_size=16, sample_interval=100,path = "gdrive/My Drive/anime2/")

W0813 06:02:34.024345 140582191855488 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0813 06:02:42.260715 140582191855488 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0813 06:02:42.261716 140582191855488 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:181: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.



2019-08-13 06:02:46.298909


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:115: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:92: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:93: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
/usr/local/lib/python3.6/dist-packages/keras/engine/

[Epoch 0/80] [Batch 0/256] [D loss: 0.280455, acc:  56%] [G loss: 3.185999, adv: 0.619625, recon: 0.092421, id: 0.062726] time: 0:00:42.570362 


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:115: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:29: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:92: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:93: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weigh

[Epoch 0/80] [Batch 1/256] [D loss: 0.303383, acc:  58%] [G loss: 5.065341, adv: 0.852081, recon: 0.159577, id: 0.089871] time: 0:00:45.875260 
[Epoch 0/80] [Batch 2/256] [D loss: 0.232240, acc:  63%] [G loss: 4.138973, adv: 0.810755, recon: 0.117845, id: 0.092100] time: 0:00:48.109113 
[Epoch 0/80] [Batch 3/256] [D loss: 0.295961, acc:  51%] [G loss: 4.220382, adv: 0.836440, recon: 0.121792, id: 0.066765] time: 0:00:50.343316 
[Epoch 0/80] [Batch 4/256] [D loss: 0.180418, acc:  73%] [G loss: 4.403722, adv: 0.911502, recon: 0.122135, id: 0.074087] time: 0:00:52.580981 
[Epoch 0/80] [Batch 5/256] [D loss: 0.205977, acc:  68%] [G loss: 4.072976, adv: 0.953348, recon: 0.102560, id: 0.069863] time: 0:00:54.821031 
[Epoch 0/80] [Batch 6/256] [D loss: 0.231657, acc:  63%] [G loss: 4.116142, adv: 1.014802, recon: 0.098874, id: 0.056720] time: 0:00:57.075624 
[Epoch 0/80] [Batch 7/256] [D loss: 0.250959, acc:  59%] [G loss: 4.036689, adv: 0.954962, recon: 0.100329, id: 0.080918] time: 0:00:59.